In [1]:
import time
import logging
import concurrent.futures
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from datetime import date, timedelta
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

In [2]:
def get_weather_data_for_dates7(current_date, end_date, num_threads=2):
    weather_data = {}
    problematic_urls = []
    #current_date = datetime(start_year, start_month, start_day)
    #end_date = datetime(end_year, end_month, end_day)
    dates = []
    while current_date <= end_date:
        dates.append(current_date)
        current_date += timedelta(days=1)

    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        results = [executor.submit(get_weather_data, date.year, date.month, date.day) for date in dates]

        for result in results:
            date_data = result.result()
            weather_data[date_data[0]] = date_data[1]

    return weather_data, problematic_urls

In [ ]:
def get_weather_data(year, month, day):
    try:
        browser = webdriver.Firefox()
        browser.get('https://www.wunderground.com/history/daily/il/haifa/LLHA')
        WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.ID, 'yearSelection')))
        
        year_select = Select(browser.find_element(By.ID,'yearSelection'))
        year_select.select_by_value(str((2023-year))+':'+' '+str(year))
        month_select = Select(browser.find_element(By.ID,'monthSelection'))
        month_select.select_by_value(str(month))
        day_select = Select(browser.find_element(By.ID,'daySelection'))
        day_select.select_by_value(str((day-1))+':'+' '+str(day))

        browser.find_element(By.ID, 'dateSubmit').click()
        WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.ID, 'history-ob')))
        
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        data_rows = soup.find_all('tr', {'class': 'no-metars'})
        date_data = {}